In [8]:
import pandas as pd

def correct_population_dataset(
    file_path,
    canada_population_file,
    provinces_age_gender_file,
    canada_age_gender_file,
    output_path=None
):
    # Load base demographic data
    df = pd.read_csv(file_path)
    df['Province'] = df['Province'].str.strip()
    df_unique = df.drop_duplicates(subset=["Year", "Province"])
    df_unique = df_unique[df_unique["Year"] >= 1991]

    # Load official Canada population
    canada_population_df = pd.read_csv(canada_population_file)
    canada_population_df = canada_population_df[canada_population_df["Year"].isin(df_unique["Year"].unique())]

    common_columns = ["Year", "Province", "Total PRs", "Total TRs", "Total Births", "Total Deaths", "Population Estimate"]
    df_unique = df_unique[common_columns]
    canada_population_df = canada_population_df[common_columns]

    # Recalculate 'Canada' totals
    df_corrected = df_unique[df_unique["Province"] != "Canada"]
    grouped_df = df_corrected.groupby('Year').sum().reset_index()
    grouped_df['Province'] = 'Canada'

    canada_population_df = canada_population_df.set_index('Year')
    grouped_df = grouped_df.set_index('Year')
    for col in ['Total PRs', 'Total TRs', 'Total Births', 'Total Deaths']:
        canada_population_df[col] = grouped_df[col].combine_first(canada_population_df[col])
    canada_population_df = canada_population_df.reset_index()

    df_final = pd.concat([df_corrected, canada_population_df], ignore_index=True)
    df_final = df_final.sort_values(by=["Year", "Province"]).reset_index(drop=True)

    # Clean Age-Gender Data
    def clean_age_gender(df):
        df = df[['REF_DATE', 'GEO', 'Gender', 'Age group', 'VALUE']].copy()
        df.columns = ['Year', 'Province', 'Gender', 'Age_Group', 'Population']
        df['Province'] = df['Province'].str.strip()
        df = df[df['Gender'].isin(['Men+', 'Women+'])].dropna(subset=['Population'])
        df['Year'] = pd.to_datetime(df['Year'].astype(str) + '-01-01')
        df['REF_DATE'] = df['Year'].dt.strftime('01-%m-%Y')
        df['Year'] = df['Year'].dt.year
        df = df[df['Year'] >= 1991]
        return df

    provinces_df = pd.read_csv(provinces_age_gender_file)
    canada_df = pd.read_csv(canada_age_gender_file)
    provinces_clean = clean_age_gender(provinces_df)
    canada_clean = clean_age_gender(canada_df)
    combined = pd.concat([provinces_clean, canada_clean], ignore_index=True)

    # Pivot to Male/Female columns
    gender_pivoted = combined.pivot_table(
        index=['REF_DATE', 'Year', 'Province', 'Age_Group'],
        columns='Gender',
        values='Population',
        aggfunc='sum'
    ).reset_index().rename(columns={'Men+': 'Male_Population', 'Women+': 'Female_Population'})

    # Merge with base demographic totals
    merged = pd.merge(gender_pivoted, df_final, how='left', on=['Year', 'Province'])

    # Interpolate for Canada
    canada_mask = merged['Province'] == 'Canada'
    numeric_cols = ['Male_Population', 'Female_Population', 'Total PRs', 'Total TRs',
                    'Total Births', 'Total Deaths', 'Population Estimate']
    for col in numeric_cols:
        merged.loc[canada_mask, col] = merged.loc[canada_mask, col].interpolate(method='linear', limit_direction='both')

    # Map age groups into broader categories
    def map_age_group(age_group):
        if age_group in ['0 to 14 years', '0 to 15 years', '0 to 16 years', '0 to 17 years']:
            return 'Children (0–17)'
        elif age_group in ['15 to 49 years']:
            return 'Youth & Adults (15–49)'
        elif age_group in ['15 to 64 years', '16 to 64 years', '17 to 64 years']:
            return 'Working Age (15–64)'
        elif age_group == '18 years and older':
            return 'Adults (18+)'
        elif age_group == '90 years and older':
            return 'Elderly (90+)'
        elif age_group == 'All ages':
            return 'All Ages'
        else:
            return 'Other/Unmapped'

    merged['Age_Group_Category'] = merged['Age_Group'].apply(map_age_group)
    merged = merged[merged['Age_Group_Category'] != 'Other/Unmapped']

    # Group by category using mean to avoid overcounting
    agg_cols = ['REF_DATE', 'Year', 'Province', 'Age_Group_Category']
    merged_grouped = merged.groupby(agg_cols)[numeric_cols].mean().reset_index()

    # Fill missing values using 'All Ages' rows
    all_ages = merged_grouped[merged_grouped['Age_Group_Category'] == 'All Ages'][['Year', 'Province', 'Total Births', 'Total Deaths', 'Population Estimate']]
    merged_grouped = pd.merge(merged_grouped, all_ages, on=['Year', 'Province'], how='left', suffixes=('', '_filled'))

    for col in ['Total Births', 'Total Deaths', 'Population Estimate']:
        merged_grouped[col] = merged_grouped[col].fillna(merged_grouped[f"{col}_filled"])
        merged_grouped.drop(columns=[f"{col}_filled"], inplace=True)

    # Final sort
    merged_grouped = merged_grouped.sort_values(by=['REF_DATE', 'Province']).reset_index(drop=True)

    # Export
    if output_path:
        merged_grouped.to_csv(output_path, index=False)
        print(f"✅ Final cleaned dataset saved to {output_path}")

    return merged_grouped


# === Usage ===
file_path = "D:/Personal Projects/IRCC_Project/datasets/Population/Metadata/Population Timeseries.csv"
canada_population_file = "D:/Personal Projects/IRCC_Project/datasets/Population/Metadata/canada_population_data.csv"
provinces_age_gender_file = "D:/Personal Projects/IRCC_Project/datasets/Population/Metadata/Demographic Data/Age Gender Distribution Provinces.csv"
canada_age_gender_file = "D:/Personal Projects/IRCC_Project/datasets/Population/Metadata/Demographic Data/Age Gender Distribution Canada.csv"
output_csv = "D:/Personal Projects/IRCC_Project/datasets/Population/Population_Demographics_by_Year_and_Province_and_Canada.csv"

final_df = correct_population_dataset(
    file_path,
    canada_population_file,
    provinces_age_gender_file,
    canada_age_gender_file,
    output_path=output_csv
)
print("✅ Data correction, cleaning, and merging completed.")


✅ Final cleaned dataset saved to D:/Personal Projects/IRCC_Project/datasets/Population/Population_Demographics_by_Year_and_Province_and_Canada.csv
✅ Data correction, cleaning, and merging completed.


In [9]:
# load the correct dataset and check for canada_population
final_df.head(20)

,REF_DATE,Year,Province,Age_Group_Category,Male_Population,Female_Population,Total PRs,Total TRs,Total Births,Total Deaths,Population Estimate
0,01-01-1991,1991,Alberta,Adults (18+),9.384580e+05,9.358570e+05,0.0,0.0,256630.0,28902.0,2572947.0
1,01-01-1991,1991,Alberta,All Ages,1.306905e+06,1.285401e+06,0.0,0.0,256630.0,28902.0,2572947.0
2,01-01-1991,1991,Alberta,Children (0–17),3.409082e+05,3.233455e+05,0.0,0.0,256630.0,28902.0,2572947.0
3,01-01-1991,1991,Alberta,Elderly (90+),2.183000e+03,4.900000e+03,0.0,0.0,256630.0,28902.0,2572947.0
4,01-01-1991,1991,Alberta,Working Age (15–64),8.737843e+05,8.396293e+05,0.0,0.0,256630.0,28902.0,2572947.0
5,01-01-1991,1991,Alberta,Youth & Adults (15–49),7.385780e+05,7.090010e+05,0.0,0.0,256630.0,28902.0,2572947.0
6,01-01-1991,1991,British Columbia,Adults (18+),1.269947e+06,1.299830e+06,0.0,0.0,273628.0,47954.0,3339935.0
7,01-01-1991,1991,British Columbia,All Ages,1.681631e+06,1.692156e+06,0.0,0.0,273628.0,47954.0,3339935.0
8,01-01-1991,1991,British Columbia,Children (0–17),3.785575e+05,3.609455e+05,0.0,0.0,273628.0,47954.0,3339935.0
9,01-01-1991,1991,British Columbia,Elderly (90+),3.598000e+03,9.040000e+03,0.0,0.0,273628.0,47954.0,3339935.0


In [37]:
corrected_df.head(20)

,Year,Province,Total PRs,Total TRs,Total Births,Total Deaths,Population Estimate
0,1991,Alberta,0,0,256630,28902,2572947
1,1991,British Columbia,0,0,273628,47954,3339935
2,1991,Canada,0,0,2413304,391138,27790000
3,1991,Manitoba,0,0,103642,17886,1106196
4,1991,New Brunswick,0,0,56978,10938,743210
5,1991,Newfoundland and Labrador,0,0,42968,7596,577377
6,1991,Northwest Territories including Nunavut,0,0,9812,474,59711
7,1991,Nova Scotia,0,0,72112,14510,912792
8,1991,Ontario,0,0,909316,145834,10355101
9,1991,Prince Edward Island,0,0,11296,2376,130477


In [38]:
def calculate_metrics(df):
    # Initialize new columns with default value 0
    df['Net Migration Rate'] = 0
    df['Natural Growth Rate'] = 0
    df['Net Migration'] = 0
    df['Natural Increase'] = 0
    df['Net Population Change'] = 0
    df['Population Growth Rate (%)'] = 0

    for index, row in df.iterrows():
        if row['Total PRs'] != 0 and row['Total TRs'] != 0:
            df.at[index, 'Net Migration'] = row['Total PRs'] - row['Total TRs']
        
        if row['Total Births'] != 0 and row['Total Deaths'] != 0:
            df.at[index, 'Natural Increase'] = row['Total Births'] - row['Total Deaths']
        
        if row['Population Estimate'] != 0:
            if df.at[index, 'Net Migration'] != 0:
                df.at[index, 'Net Migration Rate'] = (df.at[index, 'Net Migration'] / row['Population Estimate']) * 1000
            
            if df.at[index, 'Natural Increase'] != 0:
                df.at[index, 'Natural Growth Rate'] = (df.at[index, 'Natural Increase'] / row['Population Estimate']) * 1000
            
            if df.at[index, 'Net Migration'] != 0 or df.at[index, 'Natural Increase'] != 0:
                df.at[index, 'Net Population Change'] = df.at[index, 'Net Migration'] + df.at[index, 'Natural Increase']
            
            if df.at[index, 'Net Population Change'] != 0:
                df.at[index, 'Population Growth Rate (%)'] = (df.at[index, 'Net Population Change'] / row['Population Estimate']) * 100

    return df

# Calculate metrics
corrected_df = calculate_metrics(corrected_df)
corrected_df.head(20)

,Year,Province,Total PRs,Total TRs,Total Births,Total Deaths,Population Estimate,Net Migration Rate,Natural Growth Rate,Net Migration,Natural Increase,Net Population Change,Population Growth Rate (%)
0,1991,Alberta,0,0,256630,28902,2572947,0.0,88.508625,0,227728,227728,8.850862
1,1991,British Columbia,0,0,273628,47954,3339935,0.0,67.568381,0,225674,225674,6.756838
2,1991,Canada,0,0,2413304,391138,27790000,0.0,72.765959,0,2022166,2022166,7.276596
3,1991,Manitoba,0,0,103642,17886,1106196,0.0,77.523332,0,85756,85756,7.752333
4,1991,New Brunswick,0,0,56978,10938,743210,0.0,61.947498,0,46040,46040,6.194750
5,1991,Newfoundland and Labrador,0,0,42968,7596,577377,0.0,61.263265,0,35372,35372,6.126326
6,1991,Northwest Territories including Nunavut,0,0,9812,474,59711,0.0,156.386595,0,9338,9338,15.638660
7,1991,Nova Scotia,0,0,72112,14510,912792,0.0,63.105286,0,57602,57602,6.310529
8,1991,Ontario,0,0,909316,145834,10355101,0.0,73.730039,0,763482,763482,7.373004
9,1991,Prince Edward Island,0,0,11296,2376,130477,0.0,68.364539,0,8920,8920,6.836454


In [39]:
corrected_df.to_csv("D:/Personal Projects/IRCC_Project/datasets/Population/Population_Metrics_by_Year_and_Province.csv", index=False)
print("Dataset with metrics saved successfully!")

Dataset with metrics saved successfully!
